## Большие языковые модели (используем google colab на максимум)

![img](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F4470ce74-e595-4750-92a5-5f21f040df6d_577x432.jpeg)

## Часть 1. Инжиниринг подсказок (prompt engeneering)

В задании мы будем использовать общедоступные API-интерфейсы, моделей у которых 100+ миллиардов весов для логического вывода. Ваша задача — подтолкнуть модель к решению нескольких задач за вас.

- Если у вас есть доступ к ChatGPT от openAI используйте его [(чятикГПТ)](https://chat.openai.com/chat)

- Если нет, используйте BLOOM API [(БЛУМ)](https://huggingface.co/bigscience/bloom)

**Задача 1.**

Заставьте модель сгенерировать диалог двух персон на выбор:
- Известная личность или политический деятель
- Вымышленный персонаж (из литературы, кино и тд.)
- Лично вы

Сравните два случая: 

1) подсказка содержла только имена

2) подсказка содержала дополнительную информацию (см. пример ниже)


![img](https://i.imgur.com/a1QhKF7.png)

**Задача 2.**

Используйте zero-shot prompt (без примеров), чтобы перевести отрывок монолога гамлета с русского на английский.

```
To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them. To die—to sleep,
No more; and by a sleep to say we end
The heart-ache and the thousand natural shocks
That flesh is heir to: 'tis a consummation
Devoutly to be wish'd. To die, to sleep;
To sleep, perchance to dream—ay, there's the rub:
For in that sleep of death what dreams may come,
When we have shuffled off this mortal coil,
Must give us pause—there's the respect
That makes calamity of so long life.
```

**Задача 3.**
Создайте быстрые и короткие подсказки, которые заставят модель изменить родовые местоимения главного действующего лица в данном предложении в любом направлении по вашему выбору. Например: doctor took off his mask <-> the doctor took of her mask.


**Задача 4**

Напишите подсказку и предоставьте примеры, чтобы модель преобразовывала имперские единицы в метрические единицы (мили -> километры; мили в час -> км/ч). Учтите что модель должна переводить значения а не только менять единицу измерения, т.е. 1 mile -> 1.6 km а не 1 mile -> 1 km (Модель не обязательно должна точно переводить, но хотя бы примерно правильно. Скажем с точностью до целых).

После того, как он работает с базовыми расстояниями и скоростью, попробуйте найти сложные примеры, где он не работает.

## Fine-tunning больших языковых моделей.

Коллаб предоставляет доступ к GPU, для этого нужно поменять runtime. 
<center><img src="https://i.imgur.com/OOfDYzJ.png" width=240px></center>

Теперь давайте попробуем загрузить уменьшенную версию OPT без API. Мы будем использовать OPT-6b7 с 6,7 млрд параметров. Осторожно: хотя эта модель меньше, чем модели в API, она все же более чем в 60 раз больше, чем BERT. Приведенный ниже код едва помещается в память, поэтому убедитесь, что у вас больше ничего не загружено. Иногда вам нужно перезапустить среду выполнения, чтобы код заработал.

In [ ]:
import torch
if torch.cuda.get_device_capability() < (7, 5):
  raise ValueError(f"You got a GPU with capability {torch.cuda.get_device_capability()}, need at least (7, 5)")
else: print("OK")

# Note: this code requires a Turing GPU or newer. Good: T4, RTX 20xx/30xx, A100/Axx; Bad: K80, P100, V100
# Colab gives you T4. If you get older GPUs, please wait or switch to a new account (don't use both at the same time)
%pip install --quiet bitsandbytes==0.35.4 transformers==4.24.0 datasets==2.7.0 accelerate==0.14.0

Ниже приведено много кода. Чтобы в нем разобраться нужно пройти следющие курсы в университете по порядку: ML, DL, NLP. А для этих курсов желательно уже знать матанализ, теорвер и линейную алгебру. 

В общем просто запустите при желании этот код, чтобы посмотреть на что в пределе способем google colab. Модель займет всю оперативку и обучать на видеокарте её надо минимум 3 часа.

Что происходит: мы взяли большую языковую модель, предобученную на твиттере. После этого мы её дообучаем на корпусе с [английскими цитатами](https://huggingface.co/datasets/Abirate/english_quotes). После этого модель сможет правдоподобно генерировать цитаты знаменитых людей

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-6.7b", load_in_8bit=True, device_map='auto',
    low_cpu_mem_usage=True, torch_dtype=torch.float16, offload_state_dict=True)
# note: these flags slow down the code to save RAM; remove them if you have >32GB RAM
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")

for module in model.modules():
    if isinstance(module, bnb.nn.Linear8bitLt):
        module.state.memory_efficient_backward = True

for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.model.decoder.project_in = lambda x: x.requires_grad_(True)

# cast model outputs to float32 to unfuck the top-k sampler
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
old_lm_head = model.lm_head
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
batch = tokenizer("Mark Zuckerberg is", return_tensors='pt')
# note to self: find a less controversial example

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, min_length=30, max_length=30, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0].numpy()))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))

    def forward(self, input):
        return self.module(input) + torch.matmul(torch.matmul(input, self.adapter_A), self.adapter_B)


In [ ]:
# test your implementation
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LoRALayer(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "please check your forward pass"

test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128), torch.linspace(-1, 1, 128).unsqueeze(0))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)
dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "bad grad w.r.t. A"
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "bad grad w.r.t. B"
# note: bad grad means that your code is different from LoRA paper OR that your code is not autograd-friendly (e.g. no_grad)
del dummy_loss, test_linear, test_adapter
print("All tests passed!")

In [ ]:
for name, module in model.named_modules():
  if 'OPTAttention' in repr(type(module)):
    module.q_proj = LoRALayer(module.q_proj, rank=8)
    module.k_proj = LoRALayer(module.k_proj, rank=8)
    module.v_proj = LoRALayer(module.v_proj, rank=8)

In [ ]:
batch = tokenizer("Mark Zuckerberg is", return_tensors='pt')
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast():
  out = model.forward(**batch)
  out.logits.norm().backward()

for module in model.modules():
  if isinstance(module, LoRALayer):
    assert module.adapter_B.grad is not None
    assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)

In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

trainer = transformers.Trainer(
    model=model, train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, gradient_accumulation_steps=4,
        warmup_steps=250, max_steps=500, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs'),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Посмотрим на результат:

In [ ]:
batch = tokenizer("If you want to be rich", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, min_length=30, max_length=200, do_sample=True)

print('\n\n', tokenizer.decode(output_tokens[0].numpy()))